In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime
import os
import base64
import requests

# Initialize CoinGecko
cg = CoinGeckoAPI()

# Define coins
coins = {
    'BTC': 'bitcoin',
    'ETH': 'ethereum',
    'ADA': 'cardano',
    'SOL': 'solana',
    'DOT': 'polkadot',
    'AVAX': 'avalanche-2'
}

# --- GitHub + Airtable Configuration ---
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
BASE_ID = "appnssPRD9yeYJJe5"
TABLE_NAME = "Database"
airtable_url = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"
airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}",
    "Content-Type": "application/json"
}

# --- Fetch existing record from Airtable ---
response = requests.get(airtable_url, headers=airtable_headers)
response.raise_for_status()
data_airtable = response.json()

# Initialize record_id and existing_data
record_id = None
existing_data = []

# Get the existing data from Airtable (for this example, assume 365 days data)
for record in data_airtable['records']:
    if record['fields'].get('Name') == "365-Day Volatility and Range":
        existing_data = record['fields'].get('Data', [])  # Field that holds the data
        record_id = record['id']
        break

# --- Fetch 365 days of OHLC data from CoinGecko and compute metrics ---
new_data = []

for symbol, coin_id in coins.items():
    market_data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency='usd', days=365)
    ohlc_data = market_data['prices']
    
    for i in range(1, len(ohlc_data)):
        prev_day = ohlc_data[i-1]
        current_day = ohlc_data[i]
        
        prev_timestamp, prev_price = prev_day
        current_timestamp, current_price = current_day
        
        high = max(prev_price, current_price)
        low = min(prev_price, current_price)
        
        volatility = ((high - low) / low) * 100 if low > 0 else 0
        trading_range = high - low
        
        new_data.append({
            'symbol': symbol,
            'timestamp': datetime.utcfromtimestamp(current_timestamp / 1000),
            'high_24h_usd': high,
            'low_24h_usd': low,
            'volatility_24h_%': round(volatility, 2),
            'trading_range_24h_usd': round(trading_range, 2)
        })

# --- Check for new data that isn't in the existing data ---
new_data_to_append = []
if existing_data:
    # If the record exists, find new data
    for new_entry in new_data:
        is_duplicate = any(
            existing['timestamp'] == new_entry['timestamp'] and
            existing['symbol'] == new_entry['symbol']
            for existing in existing_data
        )
        if not is_duplicate:
            new_data_to_append.append(new_entry)
else:
    # If no existing data, append all new data
    new_data_to_append = new_data

# --- If new data exists, append it to Airtable ---
if new_data_to_append:
    # Merge new data with existing data (if any)
    existing_data.extend(new_data_to_append)

    # Prepare payload to update the Airtable record
    patch_payload = {
        "fields": {
            "Name": "365-Day Volatility and Range",
            "Data": existing_data  # Append new data to existing data
        }
    }

    # If the record exists, update it
    if record_id:
        patch_url = f"{airtable_url}/{record_id}"
        airtable_resp = requests.patch(patch_url, headers=airtable_headers, json=patch_payload)

        if airtable_resp.status_code == 200:
            print("✅ Successfully updated Airtable with new data.")
        else:
            print(f"❌ Failed to update Airtable: {airtable_resp.status_code} - {airtable_resp.text}")
    else:
        # If no record found, create a new one
        post_payload = {
            "records": [{
                "fields": {
                    "Name": "365-Day Volatility and Range",
                    "Data": existing_data
                }
            }]
        }
        airtable_resp = requests.post(airtable_url, headers=airtable_headers, json=post_payload)

        if airtable_resp.status_code == 200:
            print("✅ Successfully created new Airtable record with new data.")
        else:
            print(f"❌ Failed to create Airtable record: {airtable_resp.status_code} - {airtable_resp.text}")
else:
    print("⚠️ No new data to append. Everything is up to date.")

# --- GitHub Upload Configuration ---
GITHUB_REPO = "SagarFieldElevate/DatabaseManagement"
BRANCH = "main"
UPLOAD_PATH = "uploads"
GITHUB_TOKEN = os.getenv("GH_TOKEN")

# --- Excel Export ---
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"historical_volatility_trading_range_365_days_{timestamp}.xlsx"
df = pd.DataFrame(new_data_to_append)  # Only upload new data to GitHub
df.to_excel(filename, index=False)

# --- Upload to GitHub ---
with open(filename, "rb") as f:
    content = base64.b64encode(f.read()).decode()

upload_url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{UPLOAD_PATH}/{filename}"
upload_payload = {
    "message": f"Upload 365-day volatility and trading range {timestamp}",
    "content": content,
    "branch": BRANCH
}
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}
upload_resp = requests.put(upload_url, headers=headers, json=upload_payload)

if upload_resp.status_code not in [200, 201]:
    raise Exception(f"❌ GitHub upload failed: {upload_resp.status_code} - {upload_resp.text}")

# --- Delete the file from GitHub after upload ---
delete_url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{UPLOAD_PATH}/{filename}"
file_sha = upload_resp.json()['content']['sha']  # Get the SHA of the uploaded file
delete_payload = {
    "message": f"Delete 365-day volatility and trading range file {timestamp}",
    "sha": file_sha
}

delete_resp = requests.delete(delete_url, headers=headers, json=delete_payload)
if delete_resp.status_code != 200:
    raise Exception(f"❌ GitHub file deletion failed: {delete_resp.status_code} - {delete_resp.text}")

# --- Delete Local File ---
os.remove(filename)
print("🧽 Local file and GitHub file deleted.")
